In [1]:
import pickle
import numpy as np
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input, EfficientNetV2B2

from skimage.transform import resize

from util import TQDMPredictCallback

In [2]:
model = EfficientNetV2B2(include_top=False, weights='imagenet', input_shape=(140,140,3))

In [3]:
model.summary()

Model: "efficientnetv2-b2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 140, 140, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 140, 140, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 140, 140, 3)  0           ['rescaling[0][0]']              
                                                                                                  
 stem_conv (Conv2D)             (None, 70, 70, 32)   864         ['normalization[0

 block2c_add (Add)              (None, 35, 35, 32)   0           ['block2c_project_bn[0][0]',     
                                                                  'block2b_add[0][0]']            
                                                                                                  
 block3a_expand_conv (Conv2D)   (None, 18, 18, 128)  36864       ['block2c_add[0][0]']            
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, 18, 18, 128)  512        ['block3a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3a_expand_activation (Act  (None, 18, 18, 128)  0          ['block3a_expand_bn[0][0]']      
 ivation)                                                                                         
          

 lization)                                                                                        
                                                                                                  
 block4b_expand_conv (Conv2D)   (None, 9, 9, 416)    43264       ['block4a_project_bn[0][0]']     
                                                                                                  
 block4b_expand_bn (BatchNormal  (None, 9, 9, 416)   1664        ['block4b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block4b_expand_activation (Act  (None, 9, 9, 416)   0           ['block4b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block4b_d

                                                                                                  
 block4d_dwconv2 (DepthwiseConv  (None, 9, 9, 416)   3744        ['block4d_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block4d_bn (BatchNormalization  (None, 9, 9, 416)   1664        ['block4d_dwconv2[0][0]']        
 )                                                                                                
                                                                                                  
 block4d_activation (Activation  (None, 9, 9, 416)   0           ['block4d_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4d_s

                                                                                                  
 block5b_se_reshape (Reshape)   (None, 1, 1, 720)    0           ['block5b_se_squeeze[0][0]']     
                                                                                                  
 block5b_se_reduce (Conv2D)     (None, 1, 1, 30)     21630       ['block5b_se_reshape[0][0]']     
                                                                                                  
 block5b_se_expand (Conv2D)     (None, 1, 1, 720)    22320       ['block5b_se_reduce[0][0]']      
                                                                                                  
 block5b_se_excite (Multiply)   (None, 9, 9, 720)    0           ['block5b_activation[0][0]',     
                                                                  'block5b_se_expand[0][0]']      
                                                                                                  
 block5b_p

...

 block6a_expand_conv (Conv2D)   (None, 9, 9, 720)    86400       ['block5f_add[0][0]']            
                                                                                                  
 block6a_expand_bn (BatchNormal  (None, 9, 9, 720)   2880        ['block6a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6a_expand_activation (Act  (None, 9, 9, 720)   0           ['block6a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6a_dwconv2 (DepthwiseConv  (None, 5, 5, 720)   6480        ['block6a_expand_activation[0][0]
 2D)                                                             ']                               
          

 )                                                                                                
                                                                                                  
 block6c_activation (Activation  (None, 5, 5, 1248)  0           ['block6c_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block6c_se_squeeze (GlobalAver  (None, 1248)        0           ['block6c_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6c_se_reshape (Reshape)   (None, 1, 1, 1248)   0           ['block6c_se_squeeze[0][0]']     
                                                                                                  
 block6c_s

                                                                                                  
 block6e_se_reduce (Conv2D)     (None, 1, 1, 52)     64948       ['block6e_se_reshape[0][0]']     
                                                                                                  
 block6e_se_expand (Conv2D)     (None, 1, 1, 1248)   66144       ['block6e_se_reduce[0][0]']      
                                                                                                  
 block6e_se_excite (Multiply)   (None, 5, 5, 1248)   0           ['block6e_activation[0][0]',     
                                                                  'block6e_se_expand[0][0]']      
                                                                                                  
 block6e_project_conv (Conv2D)  (None, 5, 5, 208)    259584      ['block6e_se_excite[0][0]']      
                                                                                                  
 block6e_p

                                                                                                  
 block6g_project_bn (BatchNorma  (None, 5, 5, 208)   832         ['block6g_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6g_add (Add)              (None, 5, 5, 208)    0           ['block6g_project_bn[0][0]',     
                                                                  'block6f_add[0][0]']            
                                                                                                  
 block6h_expand_conv (Conv2D)   (None, 5, 5, 1248)   259584      ['block6g_add[0][0]']            
                                                                                                  
 block6h_expand_bn (BatchNormal  (None, 5, 5, 1248)  4992        ['block6h_expand_conv[0][0]']    
 ization) 

 block6j_expand_bn (BatchNormal  (None, 5, 5, 1248)  4992        ['block6j_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6j_expand_activation (Act  (None, 5, 5, 1248)  0           ['block6j_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block6j_dwconv2 (DepthwiseConv  (None, 5, 5, 1248)  11232       ['block6j_expand_activation[0][0]
 2D)                                                             ']                               
                                                                                                  
 block6j_bn (BatchNormalization  (None, 5, 5, 1248)  4992        ['block6j_dwconv2[0][0]']        
 )        

In [4]:
with open("./data/posters_resize.pickle","rb") as fr:
    posters = pickle.load(fr)

In [5]:
feature_map = model.predict(posters, batch_size=64)

In [7]:
feature_map.shape, feature_map.dtype

((26938, 5, 5, 1408), dtype('float32'))

In [8]:
with open("./data/poster_feature_map.pickle","wb") as fw:
    pickle.dump(feature_map, fw, pickle.HIGHEST_PROTOCOL)